	# Author: Alexander Staub
	## Last changed: 2025.08.26
	## Purpose: Checking the overlap between Khwan's audio char data and our data


In [8]:
#installing packages
import logging
import pandas as pd
from pprint import pprint
import numpy as np
import json
import os
import glob
from typing import Dict, Any, List, Optional
import gc  # For garbage collection
import csv

In [27]:
# loading in the relevant datasets
khwan_audio_chars = pd.read_csv("//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/Khwan_Kim/2025_08_matched_spotify_ids.csv")
charts_with_spotify = pd.read_csv("//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/Spotify/1980_2000_songs_artists/final_charted_from_spotify_1980_2000.csv")
mb_with_spotify = pd.read_csv("//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/Spotify/1980_2000_songs_artists/musicbrainz_spotify_combined_track_artist_final.csv")
charts_no_spotify = pd.read_csv("//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/Spotify/1980_2000_songs_artists/all_charts_songs_1980_2000_complete_clean.csv")
mb_songs_no_spotify = pd.read_csv("//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/musicbrainz/sql_exports/musicbrainz_data_refined/release_recordings_unique_80_10_refined_ISRC.csv")
mb_country_labels = pd.read_csv("//bigdata.wu.ac.at/delpero/Data_alexander/data/raw_data/musicbrainz/sql_exports/release_country_labels_80_10_w_artists.csv")

In [ ]:
#investigate the variables
# print(mb_songs_no_spotify.columns)
# print(mb_with_spotify.columns)
# print(khwan_audio_chars.columns)
# print(mb_country_labels.columns)

# Chart level analysis

In [10]:
# create a column (audio_chars) that is either 0/1 depending on whether column sp_tr_id in khwan_audio_chars matches with spotify_track_id 
charts_with_spotify['audio_chars'] = charts_with_spotify['spotify_track_id'].isin(khwan_audio_chars['# sp_tr_id']).astype(int)

In [17]:
# Now I want a 0/1 column in the charts_no_spotify dataframe that indicates whether a record is in charts_with_spotify and whether it has 0/1 in audio_chars
# based on the column 'tf_name_artist_credit'

charts_no_spotify['in_charts_with_spotify'] = charts_no_spotify['tf_artist_song'].isin(charts_with_spotify['tf_artist_song']).astype(int)
charts_no_spotify['audio_chars'] = charts_no_spotify['tf_artist_song'].isin(charts_with_spotify[charts_with_spotify['audio_chars'] == 1]['tf_artist_song']).astype(int)

In [ ]:
#print the column names in charts_no_spotify
# print(charts_no_spotify.columns)

## The country bias analysis
- currently no date variable because did not manage to get out of the US (CB and Billboard) charts just yet

In [ ]:
# Create the summary table
def analyze_chart_bias(df):
    """
    Analyze potential bias in Spotify ID and audio characteristics availability by country
    """
    
    # Calculate total records per country
    country_counts = df['country'].value_counts()
    total_records = len(df)
    
    # Calculate records with Spotify IDs per country
    spotify_counts = df[df['in_charts_with_spotify'] == 1]['country'].value_counts()
    
    # Calculate records with audio characteristics per country
    audio_counts = df[df['audio_chars'] == 1]['country'].value_counts()
    
    # Create summary dataframe
    summary_df = pd.DataFrame({
        'total_records': country_counts,
        'pct_of_dataset': (country_counts / total_records * 100).round(2),
        'records_with_spotify': spotify_counts.reindex(country_counts.index, fill_value=0),
        'records_with_audio': audio_counts.reindex(country_counts.index, fill_value=0)
    })
    
    # Calculate percentages within each country
    summary_df['pct_with_spotify'] = (summary_df['records_with_spotify'] / 
                                       summary_df['total_records'] * 100).round(2)
    summary_df['pct_with_audio'] = (summary_df['records_with_audio'] / 
                                    summary_df['total_records'] * 100).round(2)
    
    # Calculate overall percentages for comparison
    overall_spotify_pct = (df['in_charts_with_spotify'].sum() / len(df) * 100)
    overall_audio_pct = (df['audio_chars'].sum() / len(df) * 100)
    
    # Add difference from overall percentage (to identify bias)
    summary_df['spotify_diff_from_overall'] = (summary_df['pct_with_spotify'] - 
                                                overall_spotify_pct).round(2)
    summary_df['audio_diff_from_overall'] = (summary_df['pct_with_audio'] - 
                                             overall_audio_pct).round(2)
    
    # Sort by total records (or you could sort by any other column)
    summary_df = summary_df.sort_values('total_records', ascending=False)
    
    return summary_df, overall_spotify_pct, overall_audio_pct

In [ ]:
# Run the analysis
summary_table, overall_spotify, overall_audio = analyze_chart_bias(charts_no_spotify)

In [20]:
# Display the main comparison table
print("="*80)
print("COUNTRY BIAS ANALYSIS")
print("="*80)
print(f"\nOverall Spotify ID coverage: {overall_spotify:.2f}%")
print(f"Overall Audio Characteristics coverage: {overall_audio:.2f}%")
print("\nTop 20 countries by number of records:")
print("-"*80)

COUNTRY BIAS ANALYSIS

Overall Spotify ID coverage: 42.76%
Overall Audio Characteristics coverage: 25.72%

Top 20 countries by number of records:
--------------------------------------------------------------------------------


In [21]:
# Select columns for display
display_cols = ['total_records', 'pct_of_dataset', 'pct_with_spotify', 
                'pct_with_audio', 'spotify_diff_from_overall', 'audio_diff_from_overall']
print(summary_table[display_cols].head(20).to_string())

         total_records  pct_of_dataset  pct_with_spotify  pct_with_audio  spotify_diff_from_overall  audio_diff_from_overall
country                                                                                                                     
uk               37351           66.42             41.06           24.15                      -1.70                    -1.57
de                7747           13.78             42.11           22.49                      -0.65                    -3.23
us                6105           10.86             64.93           49.70                      22.17                    23.98
fr                3580            6.37             16.59            9.13                     -26.17                   -16.59
it                1448            2.58             61.33           23.55                      18.57                    -2.17


# The musicbrainz analysis

In [29]:
# create a column (audio_chars) that is either 0/1 depending on whether column "# sp_tr_id" in khwan_audio_chars matches with mb_with_spotify 'spotify_track_id'
mb_with_spotify['audio_chars'] = mb_with_spotify['spotify_track_id'].isin(khwan_audio_chars['# sp_tr_id']).astype(int)

In [30]:
# Now I want a 0/1 column in the mb_songs_no_spotify dataframe that indicates whether a record is in mb_with_spotify and whether it has 0/1 in audio_chars
# based on the column 'id_track'
mb_songs_no_spotify['in_mb_with_spotify'] = mb_songs_no_spotify['id_track'].isin(mb_with_spotify['id_track']).astype(int)
mb_songs_no_spotify['audio_chars'] = mb_songs_no_spotify['id_track'].isin(mb_with_spotify[mb_with_spotify['audio_chars'] == 1]['id_track']).astype(int)

In [39]:
# do a 1 to many merge of the variables 'date_year' and 'name_country' from mb_country_labels 
# into mb_songs_no_spotify based on 'id_release'
mb_songs_no_spotify_dupl = mb_songs_no_spotify.merge(mb_country_labels[['id_release', 'date_year', 'name_country']], on='id_release', how='left')


In [65]:
# Create the summary table
def analyze_mb_bias(df):
    """
    Analyze potential bias in Spotify ID and audio characteristics availability by country and year
    """
    
    # Calculate overall percentages for comparison
    overall_spotify_pct = (df['in_mb_with_spotify'].sum() / len(df) * 100)
    overall_audio_pct = (df['audio_chars'].sum() / len(df) * 100)
    
    # Create summaries for both dimensions
    summary_df_country = _create_summary_table(
        df, 
        'name_country', 
        overall_spotify_pct, 
        overall_audio_pct,
        sort_by='total_records',
        ascending=False
    )
    
    summary_df_year = _create_summary_table(
        df, 
        'date_year', 
        overall_spotify_pct, 
        overall_audio_pct,
        sort_by='index',
        ascending=True
    )
    
    return summary_df_country, summary_df_year, overall_spotify_pct, overall_audio_pct

In [66]:
# helper functions to create the summary table
def _create_summary_table(df, group_column, overall_spotify_pct, overall_audio_pct, 
                          sort_by='total_records', ascending=False):
    """
    Helper function to create a summary table for a given grouping column
    
    Parameters:
    -----------
    df : DataFrame
        Input dataframe
    group_column : str
        Column name to group by (e.g., 'name_country' or 'date_year')
    overall_spotify_pct : float
        Overall percentage of records with Spotify IDs
    overall_audio_pct : float
        Overall percentage of records with audio characteristics
    sort_by : str
        'total_records' or 'index' for sorting
    ascending : bool
        Sort order
    """
    total_records = len(df)
    
    # Calculate total records per group
    group_counts = df[group_column].value_counts()
    
    # Calculate records with Spotify IDs and audio chars per group
    spotify_counts = df[df['in_mb_with_spotify'] == 1][group_column].value_counts()
    audio_counts = df[df['audio_chars'] == 1][group_column].value_counts()
    
    # Create summary dataframe
    summary_df = pd.DataFrame({
        'total_records': group_counts,
        'pct_of_dataset': (group_counts / total_records * 100).round(2),
        'records_with_spotify': spotify_counts.reindex(group_counts.index, fill_value=0),
        'records_with_audio': audio_counts.reindex(group_counts.index, fill_value=0)
    })
    
    # Calculate percentages within each group
    summary_df['pct_with_spotify'] = (summary_df['records_with_spotify'] / 
                                      summary_df['total_records'] * 100).round(2)
    summary_df['pct_with_audio'] = (summary_df['records_with_audio'] / 
                                    summary_df['total_records'] * 100).round(2)
    
    # Add difference from overall percentage (to identify bias)
    summary_df['spotify_diff_from_overall'] = (summary_df['pct_with_spotify'] - 
                                               overall_spotify_pct).round(2)
    summary_df['audio_diff_from_overall'] = (summary_df['pct_with_audio'] - 
                                             overall_audio_pct).round(2)
    
    # Sort as specified
    if sort_by == 'index':
        summary_df = summary_df.sort_index(ascending=ascending)
    else:
        summary_df = summary_df.sort_values(sort_by, ascending=ascending)
    
    return summary_df

In [67]:
summary_table_country, summary_table_year, overall_spotify_mb, overall_audio_mb = analyze_mb_bias(mb_songs_no_spotify_dupl)

## The country Level analysis

In [60]:
# Display the main comparison table
print("="*80)
print("COUNTRY BIAS ANALYSIS")
print("="*80)
print(f"\nOverall Spotify ID coverage: {overall_spotify_mb:.2f}%")
print(f"Overall Audio Characteristics coverage: {overall_audio_mb:.2f}%")
print("\nTop 20 countries by number of records:")
print("-"*80)

COUNTRY BIAS ANALYSIS

Overall Spotify ID coverage: 59.48%
Overall Audio Characteristics coverage: 18.15%

Top 20 countries by number of records:
--------------------------------------------------------------------------------


In [64]:
# Select columns for display
display_cols = ['total_records', 'pct_of_dataset', 'pct_with_spotify', 
                'pct_with_audio', 'spotify_diff_from_overall', 'audio_diff_from_overall']
print(summary_table_country[display_cols].head(20).to_string())

                total_records  pct_of_dataset  pct_with_spotify  pct_with_audio  spotify_diff_from_overall  audio_diff_from_overall
name_country                                                                                                                       
United States         2903957           33.57             61.07           21.15                       1.59                     3.00
United Kingdom        1619632           18.72             60.39           18.91                       0.91                     0.76
Germany               1481820           17.13             54.17           16.70                      -5.31                    -1.45
France                 645862            7.47             54.30           18.68                      -5.18                     0.53
Netherlands            366418            4.24             60.48           15.37                       1.00                    -2.78
Italy                  269265            3.11             61.14           14

## The year level analysis

In [50]:
# Display the main comparison table
print("="*80)
print("YEAR BIAS ANALYSIS")
print("="*80)
print(f"\nOverall Spotify ID coverage: {overall_spotify_mb:.2f}%")
print(f"Overall Audio Characteristics coverage: {overall_audio_mb:.2f}%")
print("\nYears by number of records:")
print("-"*80)

YEAR BIAS ANALYSIS

Overall Spotify ID coverage: 59.48%
Overall Audio Characteristics coverage: 18.15%

Years by number of records:
--------------------------------------------------------------------------------


In [68]:
# Select columns for display
display_cols = ['total_records', 'pct_of_dataset', 'pct_with_spotify', 
                'pct_with_audio', 'spotify_diff_from_overall', 'audio_diff_from_overall']
print(summary_table_year[display_cols].head(20).to_string())

           total_records  pct_of_dataset  pct_with_spotify  pct_with_audio  spotify_diff_from_overall  audio_diff_from_overall
date_year                                                                                                                     
1980               40816            0.47             56.90           18.69                      -2.58                     0.54
1981               41116            0.48             56.22           18.19                      -3.26                     0.04
1982               45278            0.52             56.54           18.04                      -2.94                    -0.11
1983               45604            0.53             55.91           18.60                      -3.57                     0.45
1984               50677            0.59             56.12           20.31                      -3.36                     2.16
1985               56130            0.65             54.15           19.91                      -5.33          